In [ ]:
import os
import json
import numpy as np
import tensorflow as tf

In [ ]:
# List that holds all the grids to be used as inputs for training
x_train_grid = []
# List that holds all the positions to be used as inputs for training
x_train_position = []
# List that holds all the distances to be used as inputs for training
x_train_distance = []
# List that holds all the directions to be used as outputs for training
y_train = []

# List that holds all the grids to be used as inputs for testing
x_test_grid = []
# List that holds all the positions to be used as inputs for testing
x_test_position = []
# List that holds all the distances to be used as inputs for testing
x_test_distance = []
# List that holds all the directions to be used as outputs for testing
y_test = []

In [ ]:
# Name of directory for grids
directory_name = './data/small_grids/'

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_train_grid.append(i['gridworld'])
        x_train_position.append(i['position'])
        x_train_distance.append(i['distance'])
        y_train.append(i['direction'])
    
    # Close file socket
    f.close()

In [ ]:
# Name of directory for grids
directory_name = './data/small_grids_test/'

# iterate through all training grids
for file_name in os.listdir(directory_name):
    f = open(directory_name + file_name)
    data = json.load(f)

    # Iterate through all the data in a given grid and append their input and output values
    for i in data:
        x_test_grid.append(i['gridworld'])
        x_test_position.append(i['position'])
        x_test_distance.append(i['distance'])
        y_test.append(i['direction'])
    
    # Close file socket
    f.close()

In [ ]:
print(x_train_position)
print(y_train)

In [30]:
# Reshape the data
train_in_grid = np.reshape( x_train_grid, (-1, 30, 30) ) / 2
train_in_position = np.reshape( x_train_position, (-1, 2, 1) ) / 2
train_in_distance = np.reshape( x_train_distance, (-1,1,1) ) / 2
train_out = tf.keras.utils.to_categorical( y_train, 4 )

test_in_grid = np.reshape( x_test_grid, (-1, 30, 30) ) / 2
test_in_position = np.reshape( x_test_position, (-1, 2, 1) ) / 2
test_in_distance = np.reshape( x_test_distance, (-1, 1, 1) ) / 2
test_out = tf.keras.utils.to_categorical( y_test, 4 )

In [31]:
# Flatten the grid input
grid_input = tf.keras.layers.Input( shape = (30,30) )
flatten_grid = tf.keras.layers.Flatten()( grid_input )

In [32]:
# Flatten the position input
position_input = tf.keras.layers.Input( shape = (2,1) )
flatten_position = tf.keras.layers.Flatten()( position_input )

In [33]:
# flatten the direction inputs
distance_input = tf.keras.layers.Input( shape = (1,1) )
flatten_distance = tf.keras.layers.Flatten()( distance_input )

In [34]:
# Concatenate the grid and position into one vector which will be passed to neural network as input
final_input = tf.keras.layers.Concatenate()([flatten_grid, flatten_position, flatten_distance])

In [35]:
# Create layers for Neural Network
dense_1 = tf.keras.layers.Dense( units = 100, activation = tf.nn.relu )( final_input )
dense_2 = tf.keras.layers.Dense( units = 50, activation = tf.nn.relu )( dense_1 )
dense_3 = tf.keras.layers.Dense( units = 10, activation = tf.nn.relu )( dense_2 )
logits = tf.keras.layers.Dense( units = 4, activation = None )( dense_3 )
probabilities = tf.keras.layers.Softmax()( logits )

In [36]:
# Compile the neural network to use stochastic gradient descent as the optimizer and categorical_crossentropy as loss function
model = tf.keras.Model( inputs = [grid_input, position_input, distance_input], outputs = probabilities )
model.compile( optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [37]:
def generate_confusion_matrix( data, labels ):
    mat = [ [ 0 for i in range(4) ] for j in range(4) ]
    
    predictions = np.argmax( model.predict( data ), axis = 1 )
    
    for i in range( data[0].shape[0] ):
        mat[ labels[i] ][ predictions[i] ] += 1
    
    for i in range(4):
        print( "\t".join( [ str(c) for c in mat[i] ] ) )

In [38]:
# Test out before training
generate_confusion_matrix( [test_in_grid, test_in_position, test_in_distance], y_test )

0	0	2	0
0	0	15	0
0	0	38	0
0	0	57	0


In [39]:
# Train the model
history = model.fit( [train_in_grid, train_in_position, train_in_distance], train_out, epochs = 20 )

Epoch 1/20
17/17 [==============================] - 2s 13ms/step - loss: 1.1281 - accuracy: 0.4953
Epoch 2/20
17/17 [==============================] - 0s 9ms/step - loss: 1.0172 - accuracy: 0.5369
Epoch 3/20
17/17 [==============================] - 0s 12ms/step - loss: 1.0164 - accuracy: 0.5501
Epoch 4/20
17/17 [==============================] - 0s 8ms/step - loss: 1.0089 - accuracy: 0.5766
Epoch 5/20
17/17 [==============================] - 0s 9ms/step - loss: 1.0040 - accuracy: 0.5709
Epoch 6/20
17/17 [==============================] - 0s 16ms/step - loss: 0.9953 - accuracy: 0.5803
Epoch 7/20
17/17 [==============================] - 0s 9ms/step - loss: 0.9971 - accuracy: 0.5633
Epoch 8/20
17/17 [==============================] - 0s 11ms/step - loss: 1.0014 - accuracy: 0.5614
Epoch 9/20
17/17 [==============================] - 0s 10ms/step - loss: 0.9898 - accuracy: 0.5822
Epoch 10/20
17/17 [==============================] - 0s 15ms/step - loss: 0.9862 - accuracy: 0.5841
Epoch 11/20
1

In [40]:
# Test out after training
generate_confusion_matrix( [test_in_grid, test_in_position, test_in_distance], y_test )

0	0	0	2
0	0	2	13
0	0	7	31
0	0	2	55
